In [14]:
import pandas as pd
import glob
import time
import re


In [ ]:

oil = pd.concat(
    map(pd.read_excel, glob.glob("well_data/20*.xlsx"))
)



In [15]:
oil = pd.read_csv("North_Dakota_Well_Production_Data_2015_2023.csv")


In [16]:
oil.tail()

,ReportDate,API_WELLNO,FileNo,Company,WellName,Quarter,Section,Township,Range,County,...,Pool,Oil,Wtr,Days,Runs,Gas,GasSold,Flared,Lat,Long
1685897,2023-05-01,33089003290000,10752,"NEW MILLENNIUM RESOURCES, INC.",POLANCHEK 8-34,SENE,34.0,140.0,99.0,STK,...,TYLER A,554.0,3990.0,31.0,625,31.0,0,0.0,46.899267,-103.152135
1685898,2023-05-01,33009005030000,2507,EMPIRE NORTH DAKOTA LLC,OBERT LINSTAD 1,SESE,22.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,0.0,0.0,0.0,0,0.0,0,0.0,48.924446,-100.887483
1685899,2023-05-01,33009012640000,6347,EMPIRE NORTH DAKOTA LLC,RICE 2,SENW,26.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,23.0,411.0,30.0,0,0.0,0,0.0,48.917191,-100.876462
1685900,2023-05-01,33009011910000,5815,EMPIRE NORTH DAKOTA LLC,RICE 23-12,NWSW,23.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,0.0,0.0,0.0,0,0.0,0,0.0,48.928322,-100.881995
1685901,2023-05-01,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26.0,163.0,79.0,BOT,...,SPEARFISH/MADISON,46.0,9863.0,30.0,0,0.0,0,0.0,48.920808,-100.877109


In [17]:

oil.columns = [name.lower() for name in oil.columns]

oil["company"] = oil['company'].str.replace(',',"", regex=False)
oil["company"] = oil['company'].str.replace('.',"", regex=False)

# Prima
index =[949859,931153]
oil['quarter'].iloc[index]='SWSE'
oil['section'].iloc[index]=11.0
oil['township'].iloc[index]=146.0
oil['range'].iloc[index]=99.0
# sinclair
index=[951247]
oil['quarter'].iloc[index]='NENE'
oil['section'].iloc[index]=9.0
oil['township'].iloc[index]=153.0
oil['range'].iloc[index]=93.0

county_abbv = {
    "BIL":"Billings",
    "BOT":"Bottineau",
    "BOW":"Bowman",
    "BRK":"Burke",
    "DIV":"Divide",
    "DUN":"Dunn",
    "GV":"Golden Valley",
    "MCH":"McHenry",
    "MCK":"McKenzie",
    "MCL":"McLean",
    "MTL":"Mountrail",
    "REN":"Renville",
    "SLP":"Slope",
    "STK":"Stark",
    "WRD":"Ward",
    "WIL":"Williams"
}

oil['county_name'] = oil['county'].map(county_abbv)

oil['reportdate'] = pd.to_datetime(oil['reportdate'])
oil = oil.set_index('reportdate',drop=False)
oil.index.names = ["date"]
oil['year'] = oil.index.year
oil['month'] = oil.index.month

oil['reportdate']=[str(x)[:-12] for x in oil['reportdate']]

days_in_month = {
    1:31,
    2:28,
    3:31,
    4:30,
    5:31,
    6:30,
    7:31,
    8:31,
    9:30,
    10:31,
    11:30,
    12:31
}
oil['days_in_month'] = oil['month'].map(days_in_month)
# mergers
oil["company"] =oil['company'].str.replace('WPX ENERGY WILLISTON LLC',"DEVON ENERGY", regex=False)
oil["company"] =oil['company'].str.replace('XTO ENERGY INC',"EXXONMOBIL", regex=False)
oil["company"] =oil['company'].str.replace('BURLINGTON RESOURCES OIL & GAS COMPANY LP',"CONOCOPHILLIPS", regex=False)
oil["company"] =oil['company'].str.replace('OASIS PETROLEUM NORTH AMERICA LLC',"CHORD ENERGY", regex=False)
oil["company"] =oil['company'].str.replace('WHITING OIL AND GAS CORPORATION',"CHORD ENERGY", regex=False)
oil['oil_per_day'] = oil['oil']/oil['days_in_month']
oil['gas_per_day'] = oil['gas']/oil['days_in_month']
oil['water_per_day'] = oil['wtr']/oil['days_in_month']
oil['gassold_per_day'] = oil['gassold']/oil['days_in_month']
oil['flared_per_day'] = oil['flared']/oil['days_in_month']
oil['gas_boe'] = oil['gas']/6
oil['gas_boe_per_day'] = oil['gas_per_day']/6
oil['total_boed'] = oil['gas_boe_per_day'] + oil['oil_per_day']

# oil[oil['oil_bbl_da'].isin([np.inf, -np.inf])].index
oil = oil.dropna(0)
# oil['company'] = oil['company'].astype('category')
oil['reportdate'] = oil['reportdate'].astype('str')
# oil['wellname'] = oil['wellname'].astype('category')
# oil['quarter'] = oil['quarter'].astype('category')
# oil['county'] = oil['county'].astype('category')
# oil['county_name'] = oil['county_name'].astype('category')
# oil['fieldname'] = oil['fieldname'].astype('category')
# oil['pool'] = oil['pool'].astype('category')

# oil['fileno'] = oil['fileno'].astype('int32')
# oil['section'] = oil['section'].astype('int8')
# oil['township'] = oil['township'].astype('int16')
# oil['range'] = oil['range'].astype('int16')
# oil['oil'] = oil['oil'].astype('int64')
# oil['gas'] = oil['gas'].astype('int64')
oil['api_wellno'] = oil['api_wellno'].astype('str')
oil['month'] = oil['month'].astype('int8')
oil['year'] = oil['year'].astype('int8')
oil['days'] = oil['days'].astype('int8')
# oil['days_in_month'] = oil['days_in_month'].astype('int8')


C:\Users\matth\AppData\Local\Temp\ipykernel_15180\2715209698.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['quarter'].iloc[index]='SWSE'
C:\Users\matth\AppData\Local\Temp\ipykernel_15180\2715209698.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['section'].iloc[index]=11.0
C:\Users\matth\AppData\Local\Temp\ipykernel_15180\2715209698.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil['township'].iloc[index]=146.0

In [18]:
oil.company.unique()

array(['MUREX PETROLEUM CORPORATION', 'STATOIL OIL & GAS LP',
       'TAQA NORTH USA INC', 'HUNT OIL COMPANY', 'SM ENERGY COMPANY',
       'EOG RESOURCES INC', 'HESS BAKKEN INVESTMENTS II LLC',
       'LIME ROCK RESOURCES III-A LP', 'CHORD ENERGY',
       'SLAWSON EXPLORATION COMPANY INC', 'CRESCENT POINT ENERGY US CORP',
       'CONTINENTAL RESOURCES INC', 'EXXONMOBIL', 'BAKKEN HUNTER LLC',
       'SAMSON RESOURCES COMPANY', 'ABRAXAS PETROLEUM CORP',
       'DAVID H ARRINGTON OIL & GAS INC', 'LUFF EXPLORATION COMPANY',
       'ENERPLUS RESOURCES USA CORPORATION', 'HRC OPERATING LLC',
       'MARATHON OIL COMPANY', 'MISSOURI RIVER RESOURCES ND LLC',
       'PETROGULF CORPORATION', 'RANCH OIL COMPANY',
       'WHITE BUTTE OIL OPERATIONS LLC', 'DEVON ENERGY',
       'TRIANGLE USA PETROLEUM CORPORATION', 'ENDURO OPERATING LLC',
       'LEGACY RESERVES OPERATING LP', 'BTA OIL PRODUCERS LLC',
       'CITATION OIL & GAS CORP', 'GALAXY OIL', 'CONOCOPHILLIPS',
       'RIM OPERATING INC', 'MBI 

In [19]:
oil.tail()

,reportdate,api_wellno,fileno,company,wellname,quarter,section,township,range,county,...,month,days_in_month,oil_per_day,gas_per_day,water_per_day,gassold_per_day,flared_per_day,gas_boe,gas_boe_per_day,total_boed
date,,,,,,,,,,,,,,,,,,,,,
2023-05-01,2023-05,33089003290000,10752,NEW MILLENNIUM RESOURCES INC,POLANCHEK 8-34,SENE,34.0,140.0,99.0,STK,...,5,31,17.870968,1.0,128.709677,0.0,0.0,5.166667,0.166667,18.037634
2023-05-01,2023-05,33009005030000,2507,EMPIRE NORTH DAKOTA LLC,OBERT LINSTAD 1,SESE,22.0,163.0,79.0,BOT,...,5,31,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2023-05-01,2023-05,33009012640000,6347,EMPIRE NORTH DAKOTA LLC,RICE 2,SENW,26.0,163.0,79.0,BOT,...,5,31,0.741935,0.0,13.258065,0.0,0.0,0.000000,0.000000,0.741935
2023-05-01,2023-05,33009011910000,5815,EMPIRE NORTH DAKOTA LLC,RICE 23-12,NWSW,23.0,163.0,79.0,BOT,...,5,31,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
2023-05-01,2023-05,33009019570000,13685,EMPIRE NORTH DAKOTA LLC,RICE 3,NENW,26.0,163.0,79.0,BOT,...,5,31,1.483871,0.0,318.161290,0.0,0.0,0.000000,0.000000,1.483871


In [21]:

oil.to_pickle("ND_well_data_2015_2023_cleaned.pickle")
oil.to_csv("North_Dakota_Well_Production_Data_2015_2023_cleaned.csv", index=False)


In [27]:
oil.query('api_wellno=="33053033550000"').tail()


,reportdate,api_wellno,fileno,company,wellname,quarter,section,township,range,county,...,month,days_in_month,oil_per_day,gas_per_day,water_per_day,gassold_per_day,flared_per_day,gas_boe,gas_boe_per_day,total_boed
date,,,,,,,,,,,,,,,,,,,,,
2016-11-01,2016-11,33053033550000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,MCK,...,11,30,66.233333,105.533333,23.266667,103.800000,0.000000,527.666667,17.588889,83.822222
2016-12-01,2016-12,33053033550000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,MCK,...,12,31,43.516129,48.903226,12.935484,47.935484,0.000000,252.666667,8.150538,51.666667
2017-01-01,2017-01,33053033550000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,MCK,...,1,31,64.000000,113.870968,19.387097,78.548387,26.354839,588.333333,18.978495,82.978495
2017-02-01,2017-02,33053033550000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,MCK,...,2,28,58.357143,104.142857,18.535714,77.214286,17.500000,486.000000,17.357143,75.714286
2017-03-01,2017-03,33053033550000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,MCK,...,3,31,70.677419,101.806452,20.806452,71.225806,21.645161,526.000000,16.967742,87.645161


In [29]:
oil.query('wellname=="TEDDY FEDERAL 12X-5"')

,reportdate,api_wellno,fileno,company,wellname,quarter,section,township,range,county,...,month,days_in_month,oil_per_day,gas_per_day,water_per_day,gassold_per_day,flared_per_day,gas_boe,gas_boe_per_day,total_boed
date,,,,,,,,,,,,,,,,,,,,,
2020-05-01,2020-05,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,5,31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-06-01,2020-06,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,6,30,36.766667,45.300000,14.866667,0.000000,44.400000,226.500000,7.550000,44.316667
2020-07-01,2020-07,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,7,31,115.870968,143.903226,45.064516,0.000000,138.741935,743.500000,23.983871,139.854839
2020-08-01,2020-08,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,8,31,87.645161,108.032258,34.548387,0.000000,103.419355,558.166667,18.005376,105.650538
2020-09-01,2020-09,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,9,30,76.866667,106.300000,45.400000,69.533333,32.400000,531.500000,17.716667,94.583333
2020-10-01,2020-10,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,10,31,78.774194,121.419355,26.935484,111.258065,0.000000,627.333333,20.236559,99.010753
2020-11-01,2020-11,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,11,30,80.000000,135.500000,27.300000,124.633333,0.000000,677.500000,22.583333,102.583333
2020-12-01,2020-12,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,12,31,71.838710,123.225806,6.451613,112.483871,0.000000,636.666667,20.537634,92.376344
2021-01-01,2021-01,33025033120000,19908,EXXONMOBIL,TEDDY FEDERAL 12X-5,SWNW,5.0,148.0,97.0,DUN,...,1,31,41.935484,116.903226,13.645161,60.354839,46.161290,604.000000,19.483871,61.419355
